In [35]:
import sys
sys.path.insert(0, '../scripts')
sys.path.insert(0, '../data/database')
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
engine =  create_engine('sqlite:///../data/database/happycows.db')
with engine.connect() as con:
    weather = pd.read_sql_table('weather', con)

In [37]:
weather.head()

,index,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TMIN,TMAX
0,0,USC00361354,"CHAMBERSBURG, PA US",39.9353,-77.6394,195.1,2014-01-01,0.00,20.0,36.0
1,1,USC00361354,"CHAMBERSBURG, PA US",39.9353,-77.6394,195.1,2014-01-02,0.16,21.0,38.0
2,2,USC00361354,"CHAMBERSBURG, PA US",39.9353,-77.6394,195.1,2014-01-03,0.00,8.0,38.0
3,3,USC00361354,"CHAMBERSBURG, PA US",39.9353,-77.6394,195.1,2014-01-04,0.00,6.0,28.0
4,4,USC00361354,"CHAMBERSBURG, PA US",39.9353,-77.6394,195.1,2014-01-05,0.35,11.0,33.0


In [38]:
def transform_tmax(data):
    """fill values for TMAX"""
    data[['TMAX']] =  data.groupby('DATE')[['TMAX']].transform(lambda x: x.fillna(x.mean()))
    return data

def transform_tmin(data):
    """fill values for TMIN"""
    data[['TMIN']] =  data.groupby('DATE')[['TMIN']].transform(lambda x: x.fillna(x.mean()))
    return data

def transform_precipitation(data):
    """fill values for PRCP"""
    data[['PRCP']] =  data.groupby('DATE')[['PRCP']].transform(lambda x: x.fillna(x.mean()))
    return data

def transform_date(data):
    """ Ensure DATE is a datetime """
    data['DATE'] = pd.to_datetime(weather['DATE'])
    data = data.groupby('DATE').mean().reset_index()
    return data.set_index(['DATE'])

def transform_features(data):
    data = transform_tmax(data)
    data = transform_tmin(data)
    data = transform_precipitation(data)
    data = transform_date(data)
    return data

def drop_features(data):
    """ Drop non-essential weather features. """
    data = data.reset_index()
    return data[['DATE', 'PRCP', 'TMIN', 'TMAX']].set_index('DATE')

def identify_heatwave(data):
    """Identify a day as hot if TMAX > 90.  Identify as heatwave if three consecutive hot days"""
    data['IS_HOT'] = data['TMAX'].apply(lambda x: x > 90)
    data['IS_HEATWAVE'] = data['IS_HOT'].rolling(3).sum() == 3
    return weather

def identify_coldwave(data):
    """Identify a day as cold if TMIN < 10.  Identify as coldwave if three consecutive cold days"""
    data['IS_COLD'] = data['TMIN'].apply(lambda x: x < 10)
    data['IS_COLDWAVE'] = data['IS_COLD'].rolling(3).sum() == 3
    return weather

def make_columns_lower(data):
    """Make all column headers lowercase"""
    data = data.reset_index()
    data.columns = [x.lower() for x in data.columns]
    return data

In [39]:
weather = transform_features(weather)
weather = drop_features(weather)
weather = identify_heatwave(weather)
weather = identify_coldwave(weather)
weather = make_columns_lower(weather)
weather.head(40)

,date,prcp,tmin,tmax,is_hot,is_heatwave,is_cold,is_coldwave
0,2014-01-01,0.000000,20.0,33.5,False,False,False,False
1,2014-01-02,0.053333,22.5,38.0,False,False,False,False
2,2014-01-03,0.130000,6.0,33.0,False,False,True,False
3,2014-01-04,0.000000,3.5,21.5,False,False,True,False
4,2014-01-05,0.116667,11.5,33.0,False,False,False,False
5,2014-01-06,0.296667,14.0,42.0,False,False,False,False
6,2014-01-07,0.000000,-5.5,23.0,False,False,True,False
7,2014-01-08,0.000000,-0.5,14.5,False,False,True,False
8,2014-01-09,0.000000,13.0,29.0,False,False,False,False
9,2014-01-10,0.063333,24.5,33.5,False,False,False,False
